In [1]:
import pandas as pd 
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# Unicode, Regex, json for text digestion
import unicodedata
import re
import json

import datetime
# Time formatting
from time import strftime

from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

# nltk: natural language toolkit -> tokenization, stopwords
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer as stemmer
# nltk.download('stopwords')
import nltk.sentiment
sia = nltk.sentiment.SentimentIntensityAnalyzer()

# Quieeet!!! Y'all can't stop me now...
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jerrynolf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/jerrynolf/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
import draft_prepare as p

In [3]:
df = pd.read_csv('songs_0526.csv')
df.shape

(23762, 5)

### Preparing the data:

In [27]:
df = p.clean_df(df, extra_words = [], exclude_words = [])
df.head()

,Unnamed: 0,title,artist,date,lyrics,clean,stemmed,lemmatized,character_count,word_count,sentiment
0,2,#1,Nelly,2001-10-20,#1 LyricsUh uh uh I just gotta bring it to the...,1 lyricsuh uh uh gotta bring attention dirty t...,1 lyricsuh uh uh gotta bring attent dirti that...,1 lyricsuh uh uh gotta bring attention dirty t...,2101,396,0.9901
1,4,#9 Dream,John Lennon,1974-12-21,#9 Dream Lyrics[Verse 1] So long ago Was it in...,9 dream lyricsverse 1 long ago dream dream kno...,9 dream lyricsvers 1 long ago dream dream know...,9 dream lyricsverse 1 long ago dream dream kno...,875,152,0.9313
2,5,#Beautiful,Mariah Carey Featuring Miguel,2013-05-25,"#Beautiful Lyrics[Intro: Mariah Carey] Ah, ah,...",beautiful lyricsintro mariah carey ah ah youre...,beauti lyricsintro mariah carey ah ah your bea...,beautiful lyricsintro mariah carey ah ah youre...,973,180,0.9981
3,6,#SELFIE,The Chainsmokers,2014-03-15,#SELFIE Lyrics[Verse 1] When Jason was at the ...,selfie lyricsverse 1 jason table kept seeing l...,selfi lyricsvers 1 jason tabl kept see look gi...,selfie lyricsverse 1 jason table kept seeing l...,985,190,0.8228
4,7,#thatPOWER,will.i.am Featuring Justin Bieber,2013-04-06,#thatPOWER Lyrics[Instrumental break] [Pre-Ch...,thatpower lyricsinstrumental break prechorus j...,thatpow lyricsinstrument break prechoru justin...,thatpower lyricsinstrumental break prechorus j...,1560,279,0.9978


## Topic Modeling

#### Latent Dirichlet Allocation:

In [5]:
lemma_lyrics = df['lemmatized']
lemma_lyrics.shape[0]

23762

In [6]:
# Initialize variables
n_samples = 10_000
n_features = 2_500
n_topics = 10

# Set up dataset to be fit
dataset = lemma_lyrics
# # 
# data_samples = dataset.data[:n_samples]

# Use tf feature for LDA model
tf_vectorizer  = CountVectorizer(max_df = 1.0, min_df = 1, 
                                 max_features = n_features)
tf = tf_vectorizer.fit_transform(dataset)
# Set up LDA
lda = LatentDirichletAllocation(n_components = n_topics, max_iter = 10, random_state = 42)

lda

LatentDirichletAllocation(random_state=42)

### Other methods using Scikit-Learn

In [7]:
X_train, X_hold = train_test_split(df, test_size = .3, random_state = 42)
X_train.shape[0], X_hold.shape[0]

(16633, 7129)

In [13]:
X_train.head(1)

,Unnamed: 0,title,artist,date,lyrics,clean,stemmed,lemmatized,character_count,word_count
1576,1876,"Baby, It's Cold Outside",Glee Cast,2010-12-25,"Baby, It’s Cold Outside Lyrics[KURT & (BLAINE)...",baby cold outside lyricskurt blaine really can...,babi cold outsid lyricskurt blain realli cant ...,baby cold outside lyricskurt blaine really can...,1163,196


In [9]:
def tokenize(text):
    tokens = [word for word in nltk.word_tokenize(text) if (len(word) > 3 and len(word.strip('Xx/'))>2)]
#     stems = [stemmer.stem(letter) for letter in tokens]
    return tokens

In [10]:
tf_vectorizer = TfidfVectorizer(tokenizer = tokenize, stop_words = 'english', 
                                max_df = .75, min_df = 50, max_features = 2_500)
tf = tf_vectorizer.fit_transform(X_train.lemmatized)

In [11]:
tf.A

array([[0.        , 0.        , 0.        , ..., 0.04822434, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

AttributeError: get_names not found

### Sentiment

TypeError: 'DataFrame' object is not callable